<a href="https://colab.research.google.com/github/Daivar/Deep_Learning_Models/blob/main/CNN_hyperparemeter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision

In [ ]:
!rm -rf ./datasets/images/

!wget -q --directory-prefix='datasets/mnist-in-csv/' https://github.com/MindaugasBernatavicius/DeepLearningCourse/raw/master/07_Computer_Vision_Image_Classification/datasets/mnist-in-csv/mnist_test.csv
!wget -q --directory-prefix='datasets/mnist-in-csv/' https://github.com/MindaugasBernatavicius/DeepLearningCourse/raw/master/07_Computer_Vision_Image_Classification/datasets/mnist-in-csv/mnist_train.csv

In [ ]:
mnist_train = pd.read_csv('datasets/mnist-in-csv/mnist_train.csv')
mnist_test = pd.read_csv('datasets/mnist-in-csv/mnist_test.csv')

In [ ]:
mnist_train.head()

In [ ]:
mnist_train = mnist_train.dropna()
mnist_test = mnist_test.dropna()

In [ ]:
random_sel = mnist_train.sample(8)
random_sel.shape

In [ ]:
image_features = random_sel.drop('label', axis=1)
image_batch = (torch.Tensor(image_features.values / 255.)).reshape((-1, 28, 28))
image_batch.shape

In [ ]:
unsqueezed = image_batch.unsqueeze(1)
print(unsqueezed.shape) # torch.Size([8, 1, 28, 28]) --> batch size needs to be first dimension, so we add 1 the 2nd dimension
grid = torchvision.utils.make_grid(unsqueezed, nrow=8)
grid.shape

In [ ]:
plt.figure (figsize = (12, 12))
plt.imshow(grid.numpy().transpose((1, 2, 0)))
plt.axis('off')

In [ ]:
print(image_batch[0].max())
print(image_batch[0].min())

In [ ]:
# Identifying features and labels

mnist_train_features = mnist_train.drop('label', axis =1)
mnist_train_target = mnist_train['label']

mnist_test_features = mnist_test.drop('label', axis =1)
mnist_test_target = mnist_test['label']

In [ ]:
# converting to tensors

X_train_tensor = torch.tensor(mnist_train_features.values, dtype=torch.float)
x_test_tensor  = torch.tensor(mnist_test_features.values, dtype=torch.float) 

Y_train_tensor = torch.tensor(mnist_train_target.values, dtype=torch.long)
y_test_tensor  = torch.tensor(mnist_test_target.values, dtype=torch.long)

In [ ]:
print(X_train_tensor.shape)
print(Y_train_tensor.shape)
print(x_test_tensor.shape)
print(y_test_tensor.shape)

In [ ]:
# Reshaping the tensors according to what the CNN needs

X_train_tensor = X_train_tensor.reshape(-1, 1, 28, 28)
x_test_tensor = x_test_tensor.reshape(-1, 1, 28, 28)



In [ ]:
print(X_train_tensor.shape)
print(Y_train_tensor.shape)
print(x_test_tensor.shape)
print(y_test_tensor.shape)

In [ ]:
# Defining CNN

import torch.nn as nn
import torch.nn.functional as F



In [ ]:
# Configuring the neural network
# The input size will be the channels of the images (in_size), for RGB it will be 3, for grayscale - 1
# The final output will have a size equal to the number of classes for the prediction
# The convolving kernel will have a size of k_conv_size
# hid1 and hid2 are the kernel counts (i.e. how many feature maps there will be)

in_size = 1 # channel size
hid1_size = 16 #Re-run for 32
hid2_size = 32 #Re-run for 64
out_size = 10 # categories
k_conv_size = 5 # size of kernel

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_size, hid1_size, k_conv_size), # 24x24x16
            nn.BatchNorm2d(hid1_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)) # 12x12x16
        
        # hid1_size - is the dimension of the first convolutional layer
        # ... so the second layers needs to accept that size: Conv2d(hid1_size, ...)
        self.layer2 = nn.Sequential(
            nn.Conv2d(hid1_size, hid2_size, k_conv_size),
            nn.BatchNorm2d(hid2_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        # you can use the formula to calculate this, 
        # ... or just print our the outputs from the each layer 
        # ... and see the sizes
        self.fc = nn.Linear(512, out_size)
        
 
    def forward(self, x):
        out = self.layer1(x)
        print(out.shape) # torch.Size([55933, 16, 12, 12]) 12x12 the size of feature map produced, 16 is the depth
        out = self.layer2(out)
        print(out.shape) # torch.Size([55933, 32, 4, 4]) 4x4 the size of feature map produced, 32 is the depth
        # reshape for feeding into the FCFFNN
        out = out.reshape(out.size(0), -1)
        print(out.shape) # torch.Size([55933, 512]) 34 * 4 * 4 = 512
        out = self.fc(out)
        print(out.shape) # torch.Size([55933, 10])
        return out
        # return F.log_softmax(out, dim=-1) # we will use this latter

In [ ]:
model = ConvNet()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model.to(device)

In [ ]:
X_train_tensor = X_train_tensor.to(device)
x_test_tensor  = x_test_tensor.to(device) 

Y_train_tensor = Y_train_tensor.to(device)
y_test_tensor  = y_test_tensor.to(device)

In [ ]:
#Re-run for each different value
learning_rate = 0.001 #0.01 
criterion = nn.CrossEntropyLoss() # can work with the output of a LinearLayer as well
# criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)   

In [ ]:
# Training the model
# %%time
num_epochs = 25 # -14 min
loss_values = list()

# same training loop as before with the FCFFNN
for epoch in range(1, num_epochs):   
    outputs = model(X_train_tensor)
    loss = criterion(outputs,Y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch - %d, loss - %0.5f '%(epoch, loss.item()))
    loss_values.append(loss.item())

In [ ]:
# Model Evaluation

x = (range(0, num_epochs-1))

plt.figure(figsize = (8, 8))
plt.plot(x, loss_values)
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
# we have batch norm, so we must switch to eval mode
# ... to deactivate the batch norm
model.eval()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    
    outputs = model(x_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    
    y_test = y_test_tensor.cpu().numpy()
    predicted = predicted.cpu()
    
    print("Accuracy: ", accuracy_score(predicted, y_test))
    print("Precision: ", precision_score(predicted, y_test, average='weighted'))
    print("Recall: ", recall_score(predicted, y_test, average='weighted'))

In [ ]:
print("sample target data = ", mnist_test_target.values[1005])

In [ ]:
sample_img = mnist_test_features.values[1005]
sample_img = sample_img.reshape(1, 28, 28)

sample_img = sample_img[0, :, :]

plt.figure(figsize =(6, 6))
plt.imshow(sample_img)

In [ ]:
sample = np.array(mnist_test_features.values[1005]) 

sample_tensor = torch.from_numpy(sample).float()
sample_tensor = sample_tensor.reshape(-1, 1, 28, 28)
sample_tensor = sample_tensor.to(device)

In [ ]:
y_pred = model(sample_tensor)
y_pred

In [ ]:
_, predicted = torch.max(y_pred.data, -1)
print (" The predicted label is : ", predicted.item())

In [ ]:
!wget -q --directory-prefix='./' https://github.com/MindaugasBernatavicius/DeepLearningCourse/raw/master/07_Computer_Vision_Image_Classification/datasets/cust_img_of_3.jpg
!wget -q --directory-prefix='./' https://github.com/MindaugasBernatavicius/DeepLearningCourse/raw/master/07_Computer_Vision_Image_Classification/datasets/cust_img_of_1.jpg

In [ ]:
from PIL import Image
import PIL.ImageOps
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import skimage.transform
import skimage.io
from skimage import data

image_name = "cust_img_of_3.jpg"
image = mpimg.imread(image_name)
plt.title("Original Image")
plt.imshow(image) 

In [ ]:
original_image = Image.open(image_name)
resized_image = original_image.resize((28, 28))

plt.title("Original Image")
plt.imshow(resized_image)

In [ ]:
gray_image = skimage.color.rgb2gray(np.asarray(resized_image))
plt.imshow(gray_image, cmap='gray')
gray_image.shape

In [ ]:
inverted_image = PIL.ImageOps.invert(resized_image)
gray_image = skimage.color.rgb2gray(np.asarray(inverted_image))
plt.imshow(gray_image, cmap='gray')

In [ ]:
image_tensor = torch.from_numpy(gray_image).float()
sample_tensor = image_tensor.reshape(-1, 1, 28, 28)
sample_tensor = sample_tensor.to(device)
y_pred = model(sample_tensor)
y_pred

In [ ]:
_, predicted = torch.max(y_pred.data, -1) # the max of the prediction will be the final answer
print("The predicted label is : ", predicted.item())

In [ ]:
# Let's verify this hypothesis
for line in gray_image:
  for pixel in line:
    pixel = 1 if pixel > 0.1 else 0
    print(str(pixel).rjust(2, ' '), end='')
    # print(str(round(pixel, 2)).rjust(3, ' '), end='') # the old way does not qiute work
  print()

In [ ]:
from PIL import Image, ImageEnhance 

im = Image.open(image_name)
resized_image = im.resize((28, 28))
inverted = PIL.ImageOps.invert(resized_image)
enhancer = ImageEnhance.Contrast(inverted)
enhanced_im = enhancer.enhance(1.5) # 0.9
gray_image = skimage.color.rgb2gray(np.asarray(enhanced_im))

def increase_whites_by_threshold(img, threshold):
  for line in range(0, len(img)):
    for pixel in range(0, len(img[line])):
      img[line][pixel] = 1 if img[line][pixel] > threshold else 0

increase_whites_by_threshold(gray_image, 0.2)
plt.imshow(gray_image, cmap='gray')